# BLINK Tutorial

In [ ]:
import sys
sys.path.insert(0, '../')

import blink
import pandas as pd
from ftplib import FTP

## Step 0: Load Input Data From GNPS

#### BERKELEY-LAB.mgf Downloaded Here:
https://gnps-external.ucsd.edu/gnpslibrary

In [ ]:
gnps_host = 'massive.ucsd.edu'
gnps_user = 'anonymous'
gnps_passwd = 'a'

In [ ]:
filename = '7H12_9_1_song-75-s004-a04.mzML'    

with FTP(gnps_host, gnps_user, gnps_passwd) as ftp:
    ftp.cwd('/v02/MSV000083475/peak/mzML/Swab_PLATE7_mzML/Plate7_Samples')
    with open(filename, 'wb') as f:
        ftp.retrbinary('RETR ' + filename, f.write)

## Step 1: Read Input Data

#### BLINK can accept MGF files, mzML files, and lists of spectra as input. Correctly formatted lists of spectra and precursor m/zs can be fed directly into the discretization function, the other file formats can be parsed using a built-in helper function. 

__blink.open_msms_file():__

This function takes an MGF file or mzML file and converts into a Pandas DataFrame.

[BERKELEY-LAB.mgf](https://gnps-external.ucsd.edu/gnpslibrary)

In [ ]:
#parsing MGF:
mgf = blink.open_msms_file('BERKELEY-LAB.mgf')

#parsing mzML:
mzml = blink.open_msms_file('7H12_9_1_song-75-s004-a04.mzML')

In [ ]:
mgf.head()

In [ ]:
mzml.head()

## Step 2: "Discretize" Spectra

#### Prior to scoring, each set of spectra is discretized. This process converts lists of m/z and intensity arrays and precursor m/zs into dictionary-based sparse matrices.  

__blink.discretize_spectra():__

This function takes lists of fragmentation spectra and precursor m/zs as input and outputs a dictionary that contains intensity values, binned m/z values, and precursor m/z values. 

__Parameters:__

1. bin_width (default=0.001). This value is used to convert the m/z floats into rounded integer bins. Using smaller bin widths, precision increases and speed decreases. Example of binning calculation using default values. 100.002 --> 100002

2. intensity_power (default=0.5). The intensity power parameter scales intensity values used in the scoring. 

3. trim_empty (default=False). If True, spectra and associated metadata will be removed if the spectra are empty.

4. remove_duplicates (default=False). This parameter optionally calls blink.remove_duplicate_ions, which averages m/z values and sums the intensities of fragment ions in a spectrum that are within a minumum distance. This can be useful for spectra that are noisy or poorly centroided.

5. network_score (default=False). Optionally, BLINK can be used for molecular networking. If enabled in this step, the function will calculate another set of bins where the m/z are subtracted from the precursor m/z for that particular spectrum. This feature is still in development

8. tolerance (default=0.01). The tolerance parameter is the minumum value between fragment ion m/zs to be considered "matching" and factor into the cosine based score. However, the true tolerance of the scoring algorithm is determined by both the "tolerance" parameter and the bin width such that true tolerance is "tolerance" - "bin_width".

9. mass_diffs (default=[0]). This optional parameter allows the user to score spectra against spectra shifted by user defined chemical masses. The default value of 0 does not shift the spectra. This feature is still in development. 

In [ ]:
discretized_spectra = blink.discretize_spectra(mzml.spectrum.tolist(), mgf.spectrum.tolist(), mzml.precursor_mz.tolist(),  mgf.precursor_mz.tolist(),
                                         bin_width=0.001, tolerance=0.01, intensity_power=0.5, trim_empty=False, remove_duplicates=False, network_score=False)

Discretized spectra can be pre-computed and saved as numpy npz files using __blink.write_sparse_msms_file()__.

In [ ]:
blink.write_sparse_msms_file('sparse_spectra.npz', discretized_spectra)

Likewise, saved pre-computed sparse spectra can be read with __blink.open_sparse_msms_file()__.

In [ ]:
discretized_spectra_test = blink.open_sparse_msms_file('sparse_spectra.npz')

## Step 3: Score Spectra

#### Next, the two sets of spectra are scored against eachother. Given discretized spectra inputs, a matrix of pairwise scores is generated.

__blink.score_sprase_spectra():__

This function calculates the pairwise score and matching ion count matrices.

In [ ]:
%%time
S12 = blink.score_sparse_spectra(discretized_spectra)

In [ ]:
#cosine similarity scores
S12['mzi']

In [ ]:
#matching ion counts
S12['mzc']

## Step 4: Filter Scores

#### BLINK has several helper functions to convert the raw score and count matrices into human readable outputs. 

__blink.filter_hits()__

This function filters the score and count matrices to only include those above a user defined cutoff

__Optional Parameters:__
1. min_score (default=0.5). This is the lowest score that will be kept from the score matrix.

2. min_matches (default=5). The minumum number of matching ions to keep the score. 

3. override_matches (default=20). Keep scores with greater than or equal to this number of matches even if the score is lower than the good_score variable.

In [ ]:
filtered_S12 = blink.filter_hits(S12, min_matches=5, override_matches=20, min_score=0.6)

__blink.reformat_score_matrix():__
<br>
This is a helper function that reshapes the score/matches matrices such that they can be easily associated with the spectral metadata

In [ ]:
filtered_S12

In [ ]:
m = blink.reformat_score_matrix(filtered_S12)

In [ ]:
df = blink.make_output_df(m)

# output is stored as sparse dataframe, must to converted to standard for many operations 
df = df.sparse.to_dense()

In [ ]:
df.head()

In [ ]:
df = pd.merge(df, mgf.add_suffix("_ref"), left_on="ref", right_index=True)
df = pd.merge(df, mzml.add_suffix("_query"), left_on="query", right_index=True)

In [ ]:
df.head(20)